# Transport d'un polluant et problème adjoint

On considère le problème de transport d'un polluant modélisé à l'aide de l'équation de advection-diffusion sur une 
géométrie représentant grossièrement la surface d'une rivière surplombée de deux quais. 

\begin{align}
    -\nabla \cdot (k \nabla \phi) + \nabla \cdot (v \phi) &= 0 \qquad && \text{pour } x \in \Omega,\\
    \phi(x) &= \phi_D(x) && \text{pour } x\in \Gamma_\text{ouest}, \\
    -k\nabla \phi \cdot n &= 0  && \text{pour } x\in \partial \Omega \setminus \Gamma_\text{ouest},
\end{align}

où $\phi$ est la concentration du polluant, $k=0.01$ est le coefficient de diffusion, $v$ est le champ de vitesse.

<img src="./model_pollutant.svg" alt="Structure code Fenicsx" width="800"/>

La condition de Dirichlet sur la concentration du polluant $\phi_D(x)$ est définie ainsi:

\begin{align}
    \phi_D(x;y_0,L,c) = \begin{cases} 
        c \exp\left(\frac{1}{\left(\frac{x_2 - y_0}{L}\right)^2 -1}\right) & \text{si $\big\lvert\frac{x_2 - y_0}{L}\big\rvert < 1$}\\
                        0 & \text{sinon}
    \end{cases}
\end{align}

Cette fonction correspond à un *mollifier* centré en $y_0$ de taille $L$ et d'intensité $c$. Il est important de mentionner que cette fonction est $C^\infty(\Omega)$, que son support est $\mathbb{R} \times [y_0-L,y_0+L]$ et que toutes ses dérivées s'annulent en $x_2 \in \{y_0-L,y_0+L\}$. On fixera ces paramètres aux valeurs suivantes: $y_0 = 0.3$, $L=0.2$ et $c=1$.

Le champ de vitesse $v$ est déterminé par l'équation de Stokes

\begin{align}
    - \nu \Delta v + \nabla p &= 0 \qquad && \text{pour } x\in \Omega\\
    \nabla \cdot v & =0 && \text{pour } x\in \Omega \\
    v &=0  &&\text{pour } x\in \Gamma_\text{rive} \cup \Gamma_\text{quai}\\
    v &=v_D  &&\text{pour } x\in \Gamma_\text{ouest} \\
    (\nu \nabla v- pI) \cdot n &= 0 &&\text{pour } x\in \Gamma_\text{est} 
\end{align}

où $p$ est le champ de pression, $\nu=0.001$ est le coefficient de viscosité dynamique. La condition de Dirichlet sur le champ de vitesse $v_D$ est de la forme:
\begin{align}
    v_D(x) = -x_2(x_2-2),
\end{align}

ce qui correspond à un écoulement stationnaire entre 2 plaques. Ce problème est couplé, mais le champ de pression $p$ et de vitesse $v$ ne dépendent de la concentration $\phi$ du polluant.

On s'intéresse à la concentration du polluant les deux régions notées par $\Omega_1$ et $\Omega_2$

\begin{align}
    Q_1(\phi) = \frac{1}{\lvert\Omega_1\rvert}\int_{\Omega_1} \phi \, dx\\
    Q_2(\phi) = \frac{1}{\lvert\Omega_2\rvert}\int_{\Omega_2} \phi \, dx
\end{align}

---

## Résolution du problème de Stokes

On résoud en premier le problème de Stokes. La forme faible de ce problème mixte est: trouver $(v,p) \in W = V \times P$ tel que

\begin{align}
    \int_\Omega \nu \nabla v \cdot \nabla \beta - p \nabla \cdot \beta + q\nabla \cdot u\, dx = 0 \qquad \forall \, (\beta,q) \in W = V \times P
\end{align}

Une discrétisation stable consiste à choisir $V_h$ comme étant $(\mathcal{P}_2(\Omega))^d$ et $P_h$ comme étant $\mathcal{P}_1(\Omega)$ (éléments finis de Taylor-Hood). 

In [1]:
# Exécuter cette commande une seule fois lorsque le container est en fonctionnement. 
# La commande installe dans les modules nécessaires à la visualisation des résultats  
# %pip install ipywidgets 'pyvista[all,trame]'

import numpy as np
import pyvista
pyvista.OFF_SCREEN = True # Mettre False si l'affichage interactif semble fonctionner, sinon mettre True

import ufl # Package pour manipuler les formes bilinéaires et linéaires
from dolfinx import fem, mesh, plot # Package pour l'assemblage et pour l'affichage
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile, gmshio # Package avec des fonctions pour l'input/output de données

from mpi4py import MPI # Package pour le calcul parallèle
from petsc4py import PETSc # Package pour la résolution de

from IPython.display import display, Markdown , Latex # Modules pour l'affichage Markdown

In [2]:
(domain, cell_tags, facet_tags) = gmshio.read_from_msh("river.msh", MPI.COMM_WORLD, gdim=2)

x = ufl.SpatialCoordinate(domain)
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)
nMesh = ufl.FacetNormal(domain)

Info    : Reading 'river.msh'...
Info    : 31 entities
Info    : 22207 nodes
Info    : 44412 elements
Info    : Done reading 'river.msh'


In [3]:
pyvista.set_jupyter_backend("trame")
if pyvista.OFF_SCREEN:
    pyvista.start_xvfb()


topology, cell_types, geometry = plot.vtk_mesh(domain, 2)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    plotter.camera.zoom('tight')
    figure = plotter.screenshot("mesh.png")
    display(Markdown('<img src="./mesh.png" width="800"/>'))

<img src="./mesh.png" width="800"/>

In [4]:
nu = 0.001

def v_D(x):
    return np.stack((-x[1]*(x[1]-2),np.zeros(x.shape[1])))

def null_vel(x):
    return np.stack((np.zeros(x.shape[1]), np.zeros(x.shape[1])))

In [5]:
P2 = ufl.VectorElement("Lagrange", domain.ufl_cell(), 2)
P1 = ufl.FiniteElement("Lagrange", domain.ufl_cell(), 1)

W = fem.FunctionSpace(domain, P2 * P1)

(v, p) = ufl.TrialFunctions(W)
(beta, q) = ufl.TestFunctions(W)

### Forme bilinéaire et linéaire

La forme bilinéaire pour le problème de Stokes est

\begin{align}
    a(\{v,p\},\{\beta,q\}) = \int_\Omega \nu \nabla v \cdot \nabla \beta - p \nabla \cdot \beta + q\nabla \cdot u\, dx
\end{align}

alors que la forme linéaire est

\begin{align}
    l(\{\beta,q\}) = 0
\end{align}

In [6]:
a = fem.form((nu * ufl.inner(ufl.grad(v), ufl.grad(beta)) 
              - ufl.inner(p, ufl.div(beta)) 
              + ufl.inner(ufl.div(v), q)) * ufl.dx)

In [7]:
f = fem.Constant(domain, (0.0,0.0))
l = fem.form(ufl.inner(f,beta)*ufl.dx)

### Imposition des conditions frontières

In [8]:
W0, _ = W.sub(0).collapse()

u_dir = fem.Function(W0)
u_dir.interpolate(null_vel)
bc_dofs = fem.locate_dofs_topological((W.sub(0),W0), domain.topology.dim-1, facet_tags.find(1))
bcs1 = fem.dirichletbc(u_dir, bc_dofs, W.sub(0))

u_dir = fem.Function(W0)
u_dir.interpolate(null_vel)
bc_dofs = fem.locate_dofs_topological((W.sub(0),W0), domain.topology.dim-1, facet_tags.find(2))
bcs2 = fem.dirichletbc(u_dir, bc_dofs, W.sub(0))

u_dir = fem.Function(W0)
u_dir.interpolate(null_vel)
bc_dofs = fem.locate_dofs_topological((W.sub(0),W0), domain.topology.dim-1, facet_tags.find(4))
bcs4 = fem.dirichletbc(u_dir, bc_dofs, W.sub(0))

u_dir = fem.Function(W0)
u_dir.interpolate(v_D)
bc_dofs = fem.locate_dofs_topological((W.sub(0),W0), domain.topology.dim-1, facet_tags.find(5))
bcs5 = fem.dirichletbc(u_dir, bc_dofs, W.sub(0))


bcs = [bcs1,bcs2,bcs4,bcs5]

In [12]:
A = fem.petsc.assemble_matrix(a, bcs=bcs)
A.assemble()
b = fem.petsc.assemble_vector(l)

fem.petsc.apply_lifting(b, [a], bcs=[bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
fem.petsc.set_bc(b, bcs)

# Create and configure solver
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType("preonly")
solver.getPC().setType("lu")
solver.getPC().setFactorSolverType("superlu_dist")
# solver.setType("fgmres")
# solver.getPC().setType("ilu")
# solver.setTolerances(rtol = 1e-12, atol=1e-12, max_it=50000)

# Compute the solution
U = fem.Function(W)
solver.solve(b, U.vector)
# Split the mixed solution and collapse
v = U.sub(0).collapse()
p = U.sub(1).collapse()
v.x.scatter_forward()
p.x.scatter_forward()

In [13]:
# Enregistrement des résultats pour visualisation dans Paraview

v.name = "Vitesse"
p.name = "Pressure"
with XDMFFile(MPI.COMM_WORLD, "stokes.xdmf", "w") as xdmf:

    # Interpolation de la vitesse sur des éléments de Lagrange de degré 1 pour l'affichage
    P1_vec = ufl.VectorElement("Lagrange", domain.ufl_cell(), 1)
    v_inter = fem.Function(fem.functionspace(domain, P1_vec))
    v_inter.name = "Vitesse"
    v_inter.interpolate(v)
    
    xdmf.write_mesh(domain)
    xdmf.write_function(v_inter)
    xdmf.write_function(p)

In [14]:
topology, cell_types, geometry = plot.vtk_mesh(W0)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(v)] = v.x.array.real.reshape((geometry.shape[0], len(v)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["v"] = values
streamlines, src = function_grid.streamlines(vectors="v", n_points=50, pointa = (1.9,0.0,0.0),pointb=(1.9,2.0,0.0),return_source=True)

# Create plotter
plotter = pyvista.Plotter()
plotter.add_mesh(function_grid, style="surface",color="w")
plotter.add_mesh(streamlines)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    plotter.camera.zoom(1.5)
    figure = plotter.screenshot("stokes.png") 
    display(Markdown('<img src="./stokes.png" width="800"/>'))

<img src="./stokes.png" width="800"/>

---

## Résolution du problème de transport de polluant

Maintenant que le champ de vitesse $v$ est calculé, on peut résoudre le problème de transport de polluant. La forme faible de ce problème consiste à trouver $\phi \in V = \{v\in H^1(\Omega) \, \lvert \, u\lvert_{\Gamma_\text{Ouest}} = u_D\}$ tel que

\begin{align}
    \int_\Omega k \nabla \phi \cdot \nabla \psi + \nabla \cdot (v \phi) \psi\, dx = 0 \qquad \forall \psi \in V_0
\end{align}

où $V_0 = \{v\in H^1(\Omega) \, \lvert \, u\lvert_{\Gamma_\text{Ouest}} = 0\}$.

In [15]:
V = fem.FunctionSpace(domain, ("Lagrange", 1))
phi = ufl.TrialFunction(V)
psi = ufl.TestFunction(V)

In [16]:
k = 0.01
a_transport = fem.form((k *ufl.inner(ufl.grad(phi),ufl.grad(psi)) 
                    + ufl.div(v*phi)*psi)* ufl.dx)

In [17]:
def input_pollutant(x,y0,L,H):
    f = np.zeros(x.shape[1])
    t = 0
    for vec in x.T:
        z = (vec[1] - y0)/L
        if np.abs(z)< 1:
            f[t] = H * np.exp(1/((z)**2 - 1))
        else:
            f[t] = 0
        t = t+1

    return f

y0 = 0.3
L = 0.2
H = 1


psi_D = fem.Function(V)
psi_D.interpolate(lambda x: input_pollutant(x,y0,L,H))
facets = facet_tags.find(5) # Retourne les entités géométrique sur la frontière ouest
dofs = fem.locate_dofs_topological(V, 1, facets)
bcs = [fem.dirichletbc(psi_D, dofs)]

In [18]:
f_transport = fem.Constant(domain, 0.0)
l_transport =  fem.form(ufl.inner(f_transport,psi) * ufl.dx)

In [19]:
A = fem.petsc.assemble_matrix(a_transport, bcs=bcs)
A.assemble()

solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType("gmres")
solver.getPC().setType("ilu")
solver.setTolerances(rtol = 1e-12, atol=1e-12, max_it=500000)

b = fem.petsc.create_vector(l_transport)
with b.localForm() as loc_b:
    loc_b.set(0)
fem.petsc.assemble_vector(b, l_transport)
fem.petsc.apply_lifting(b, [a_transport], [bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
fem.petsc.set_bc(b, bcs)

phi_h = fem.Function(V)

# Solve linear problem
solver.solve(b, phi_h.vector)
phi_h.x.scatter_forward()

In [20]:
phi_h.name = "Polluant"
with XDMFFile(MPI.COMM_WORLD, "polluant.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(phi_h)

In [21]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["phi"] = phi_h.x.array.real
u_grid.set_active_scalars("phi")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=False)
u_plotter.add_mesh(streamlines)

u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
else:
    u_plotter.camera.zoom(1.5)
    figure = u_plotter.screenshot("pollutant.png")
    display(Markdown('<img src="./pollutant.png" width="800"/>'))

<img src="./pollutant.png" width="800"/>

---

# Problème adjoint et QoIs

La solution du problème adjoint indique comment le chargement influence la quantité d'intérêt. Le problème adjoint pour le problème de transport est: trouver $\lambda \in V_0$ tel que

\begin{align}
    a(\phi,\lambda) &= Q_i(\phi) \qquad &&\forall \phi \in V_0\\
    \int_\Omega k \nabla \phi \cdot \nabla \lambda + \nabla \cdot (v \phi) \lambda\, dx &= \frac{1}{\lvert \Omega_i\rvert}\int_{\Omega_i} \phi \, dx \qquad &&\forall \phi \in V_0
\end{align}

La forme forte de ce problème est obtenue en intégrant par parties:

\begin{align}
    \int_\Omega k \nabla \phi \cdot \nabla \lambda + \nabla \cdot (v \phi) \lambda - \frac{1}{\lvert \Omega_i\rvert} I_{\Omega_i}(x) \phi \, dx &= 0 \qquad &&\forall \phi \in V_0 \\
    \int_\Omega \Big(- \nabla \cdot(k\nabla \lambda) - (v\cdot\nabla\lambda) - \frac{1}{\lvert \Omega_i\rvert} I_{\Omega_i}(x) \Big)\phi \, dx + \int_{\partial \Omega} (n\cdot k \nabla \lambda) \phi  + (n \cdot v) \lambda \phi  \, ds   &= 0 \qquad &&\forall \phi \in V_0 \\
    \int_\Omega \Big(- \nabla \cdot(k\nabla \lambda) - (v\cdot\nabla\lambda) - \frac{1}{\lvert \Omega_i\rvert} I_{\Omega_i}(x) \Big)\phi \, dx + \int_{\partial \Omega \setminus \Gamma_\text{ouest}} (n\cdot k \nabla \lambda) \phi \, ds + \int_{\Gamma_\text{est}} (n \cdot v) \lambda \phi \, ds &= 0 \qquad &&\forall \phi \in V_0
\end{align}

En prenant $\phi \in V_0$ tel que $\phi=0$ sur $\partial \Omega$, et en appliquant le lemme de Du-Bois Reymond, on obtient

\begin{align}
    - \nabla \cdot(k\nabla \lambda) - (v\cdot\nabla\lambda) = \frac{1}{\lvert \Omega_i\rvert} I_{\Omega_i}(x) \qquad \text{sur } \Omega
\end{align}

On remarque que le champ de vitesse $v$ est ici inversé par rapport au problème primal initial. En prenant $\phi \in V_0$ tel que $\phi=0$ sur $\Gamma_\text{est}$, on obtient

\begin{align}
    n \cdot (k \nabla \lambda) = 0 \qquad \text{sur } \partial \Omega \setminus \Gamma_\text{ouest}
\end{align}

Une dernière condition frontière prend cette forme

\begin{align}
    (n \cdot v) \lambda = 0 \qquad \text{sur } \Gamma_\text{est}
\end{align}

Naturellement, puisque $\lambda \in V_0$, une condition de Dirichlet nulle est appliquée sur la frontière $\Gamma_\text{ouest}$.

In [22]:
a_adjoint = fem.form((k *ufl.inner(ufl.grad(psi),ufl.grad(phi)) 
                    + ufl.div(v*psi)*phi)* ufl.dx)

In [23]:
f_lambda_D = fem.Function(V)
f_lambda_D.interpolate(lambda x: 0.0*x[0])
facets = facet_tags.find(5) # Retourne les entités géométrique sur la frontière ouest
dofs = fem.locate_dofs_topological(V, 1, facets) 
bcs_adjoint = [fem.dirichletbc(f_lambda_D, dofs)]

In [24]:
Omega_1 = (1.5,0.0,2.5,0.2)
Omega_2 = (4,1.8,4.25,2.0)
Omega_3 = (4,0.8,4.25,1.0)
Omega_4 = (0.25,0.8,0.5,1.0)

Omega_qoi = Omega_3

qoi_area = (Omega_qoi[2] - Omega_qoi[0]) * (Omega_qoi[3] - Omega_qoi[1])
domain_QoI = lambda x: ufl.conditional(ufl.And(ufl.And(ufl.ge(x[0],Omega_qoi[0]),ufl.le(x[0],Omega_qoi[2])),
                                        ufl.And(ufl.ge(x[1],Omega_qoi[1]),ufl.le(x[1],Omega_qoi[3]))), 1/qoi_area, 0)

l_adjoint = fem.form(domain_QoI(x) * psi * ufl.dx)

In [25]:
A_adjoint = fem.petsc.assemble_matrix(a_adjoint, bcs=bcs_adjoint)
A_adjoint.assemble()

solver_adjoint = PETSc.KSP().create(domain.comm)
solver_adjoint.setOperators(A_adjoint)
solver_adjoint.setType("gmres")
solver_adjoint.getPC().setType("ilu")
solver_adjoint.setTolerances(rtol = 1e-12, atol=1e-12, max_it=500000)

b_adjoint = fem.petsc.create_vector(l_adjoint)
with b_adjoint.localForm() as loc_b:
    loc_b.set(0)
fem.petsc.assemble_vector(b_adjoint, l_adjoint)
fem.petsc.apply_lifting(b_adjoint, [a_adjoint], [bcs_adjoint])
b_adjoint.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
fem.petsc.set_bc(b_adjoint, bcs_adjoint)

lambda_h = fem.Function(V)

# Solve linear problem
solver_adjoint.solve(b_adjoint, lambda_h.vector)
lambda_h.x.scatter_forward()

In [26]:
lambda_h.name = "Adjoint"
with XDMFFile(MPI.COMM_WORLD, "adjoint.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(lambda_h)

In [27]:
lambda_topology, lambda_cell_types, lambda_geometry = plot.vtk_mesh(V)
lambda_grid = pyvista.UnstructuredGrid(lambda_topology, lambda_cell_types, lambda_geometry)
lambda_grid.point_data["lambda"] = lambda_h.x.array.real
lambda_grid.set_active_scalars("lambda")
lambda_plotter = pyvista.Plotter()
lambda_plotter.add_mesh(lambda_grid, show_edges=False)
# u_plotter.add_mesh(streamlines)

lambda_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    lambda_plotter.show()
else:
    lambda_plotter.camera.zoom(1.5)
    figure = lambda_plotter.screenshot("adjoint.png")
    display(Markdown('<img src="./adjoint.png" width="800"/>'))

<img src="./adjoint.png" width="800"/>